In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
counties = gpd.read_file('../Data/2020_shapefiles/tl_2020_us_county.shp')
print(counties.crs)
counties.head()

In [ ]:
type(counties)

In [ ]:
tn_county = counties.loc[counties['STATEFP'] == '47']

In [ ]:
tn_county = tn_county.reset_index(drop=True)

In [ ]:
tn_county.rename(columns={'NAME': 'county'}, inplace=True)

In [ ]:
tn_county.head(3)

In [ ]:
tn_county.plot(figsize = (12, 12));

In [ ]:
tn_county = tn_county[['county', 'geometry']]

In [ ]:
tn_county['county'] = tn_county['county'].str.title()

In [ ]:
tn_county.head(3)

In [ ]:
master_county = pd.read_csv('../Data/master_county.csv')

In [ ]:
master_county.head()

In [ ]:
# tn_county_geo = pd.merge(master_county, tn_county, how='left', left_on='county', right_on='county')

In [ ]:
tn_county_geo = master_county.merge(tn_county)

In [ ]:
tn_county_geo.info()

In [ ]:
type(tn_county_geo)

In [ ]:
county_geo = gpd.GeoDataFrame(tn_county_geo,
                           crs = tn_county.crs,
                           geometry = tn_county_geo['geometry'])

In [ ]:
type(county_geo)

In [ ]:
county_geo.info()

In [ ]:
county_geo

In [ ]:
sen20 = county_geo[(county_geo.year == 2018) & (county_geo.office == 'US_Senate')]

In [ ]:
sen20

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sen20.plot(column = 'Dem_margin', 
           cmap = 'seismic', edgecolor = 'black', 
              legend = True,
              ax = ax)
plt.title('2018 Senate Election Results', fontsize = 14, fontweight='bold')
ax.axis('off');

In [ ]:
sen06 = county_geo[(county_geo.year == 2006) & (county_geo.office == 'US_Senate')]

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sen06.plot(column = 'Dem_margin', 
              cmap = 'RdBu', vmin = -25, vmax = 25, k=8,
              edgecolor = 'black', 
              legend = True,
              ax = ax)

ax.set_facecolor('white')
fig.set_facecolor('white')
plt.title('2006 Senate Election, Democratic % Margin by County', fontsize = 14, fontweight='bold')
ax.axis('off')
# plt.savefig('senate_06.png', bbox_inches='tight')
plt.plot();

In [ ]:
gov06 = county_geo[(county_geo.year == 2006) & (county_geo.office == 'Governor')]

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
gov06.plot(column = 'Dem_margin', 
              cmap = 'RdBu', vmin = -25, vmax = 25, k=8,
              edgecolor = 'silver', 
              legend = True,
              ax = ax)
ax.set_facecolor('white')
fig.set_facecolor('white')
plt.title('2006 Gubernatorial Election, Democratic % Margin by County', fontsize = 14, fontweight='bold')
ax.axis('off')
# plt.savefig('gov_06.png', bbox_inches='tight');
plt.plot();

In [ ]:
pres04 = county_geo[(county_geo.year == 2004) & (county_geo.office == 'Pres')]
pres20 = county_geo[(county_geo.year == 2020) & (county_geo.office == 'Pres')]

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
pres04.plot(column = 'Dem_margin', 
              cmap = 'RdBu', vmin = -30, vmax = 30, k=8,
              edgecolor = 'black', 
              legend = True,
              ax = ax)
plt.title('2004 Presidential Election Results', fontsize = 14, fontweight='bold')
ax.axis('off');

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
pres20.plot(column = 'Dem_margin', 
              cmap = 'RdBu', vmin = -30, vmax = 30, k=8,
              edgecolor = 'black', 
              legend = True,
              ax = ax)
plt.title('2020 Presidential Election Results', fontsize = 14, fontweight='bold')
ax.axis('off');

In [ ]:
for index, row in county_geo.iterrows():
    if row.Rep > row.Dem:
        county_geo.loc[index, 'Rep_win'] = 1
    elif row.Rep < row.Dem:
        county_geo.loc[index, 'Rep_win'] = 0

In [ ]:
for index, row in county_geo.iterrows():
    if row.Dem > row.Rep:
        county_geo.loc[index, 'Dem_win'] = 1
    elif row.Dem < row.Rep:
        county_geo.loc[index, 'Dem_win'] = 0

In [ ]:
county_split = county_geo.groupby(['year', 'county']).agg({'Rep_win': 'sum', 'Dem_win': 'sum', 'geometry': 'first'}).reset_index()

In [ ]:
county_split.head(3)

In [ ]:
county_split = county_split.astype({'Rep_win': 'int', 'Dem_win': 'int'})

In [ ]:
for index, row in county_split.iterrows():
    if row.Rep_win == 0:
        county_split.loc[index, 'all_dem'] = 1
    elif row.Dem_win == 0:
        county_split.loc[index, 'all_rep'] = 1
    else:
        county_split.loc[index, 'split'] = 1

In [ ]:
for index, row in county_split.iterrows():
    if row.split == 1:
        county_split.loc[index, 'one_party'] = 0
    else:
        county_split.loc[index, 'one_party'] = 1

In [ ]:
county_split = county_split.fillna(0)

In [ ]:
county_split = county_split.astype({'split': 'int', 'all_dem': 'int', 'all_rep': 'int', 'one_party': 'int'})

In [ ]:
county_split.one_party.value_counts()

In [ ]:
county_split.split.value_counts()

In [ ]:
county_split

In [ ]:
split_geo = gpd.GeoDataFrame(county_split,
                           crs = tn_county.crs,
                           geometry = county_split['geometry'])

In [ ]:
type(split_geo)

In [ ]:
split_02 = split_geo[(split_geo.year == 2002)]
split_04 = split_geo[(split_geo.year == 2004)]
split_06 = split_geo[(split_geo.year == 2006)]
split_08 = split_geo[(split_geo.year == 2008)]
split_10 = split_geo[(split_geo.year == 2010)]
split_12 = split_geo[(split_geo.year == 2012)]
split_14 = split_geo[(split_geo.year == 2014)]
split_16 = split_geo[(split_geo.year == 2016)]
split_18 = split_geo[(split_geo.year == 2018)]
split_20 = split_geo[(split_geo.year == 2020)]

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_02.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2002 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_02.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_04.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2004 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_04.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_06.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2006 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_06.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_08.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2008 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_08.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_10.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2010 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_12.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2012 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_12.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_14.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2014 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_14.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_16.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2016 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_16.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_18.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2018 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_18.png', bbox_inches='tight')
plt.plot();

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))

cmap=ListedColormap(['whitesmoke','darkgoldenrod'])

split_20.plot(column = 'split', 
              cmap = cmap,
              # 'tab20c', vmin = 0, vmax = 1, k=2,
              edgecolor = 'black', 
              legend = False,
              ax = ax)
plt.title('2020 TN Election Split Ticket Counties', fontsize = 14, fontweight='bold')
ax.axis('off')
ax = plt.gca()
ax.set_facecolor('lightgrey')
fig.set_facecolor('gainsboro')
# plt.savefig('split_map_20.png', bbox_inches='tight')
plt.plot();